In [13]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Load the dataset
data = pd.read_excel("outpute.xlsx")

# Tokenize the text
data["Tokenized Text"] = data["Cleaned Text"].apply(word_tokenize)

# Save the preprocessed dataset
data.to_excel("preprocessed_output.xlsx", index=False)


In [15]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# Load the preprocessed dataset
data = pd.read_excel("preprocessed_output.xlsx")

# Train GloVe embeddings
sentences = data["Tokenized Text"].tolist()

# Save tokenized sentences to a text file
with open("sentences.txt", "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(" ".join(sentence) + "\n")

# Convert GloVe format to Word2Vec format
glove_input_file = "glove.6B.100d.txt"
word2vec_output_file = "glove.6B.100d.word2vec.txt"
glove2word2vec(glove_input_file, word2vec_output_file)

# Load the converted GloVe embeddings
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

# Save the trained embeddings
model.save("glove_model.bin")


C:\Users\Dinesh Reddy\AppData\Local\Temp\ipykernel_10604\6728923.py:18: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Load the preprocessed dataset
data = pd.read_excel("preprocessed_output.xlsx")

# Convert lists of strings to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

# Pad sequences to a fixed length
max_sequence_length = 100
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the dataset into training and testing sets
y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# Calculate evaluation metrics
report = classification_report(y_test, y_pred)
print(report)


Epoch 1/10
59/59 [==============================] - 7s 62ms/step - loss: 0.6698 - accuracy: 0.6005 - val_loss: 0.6075 - val_accuracy: 0.7042
Epoch 2/10
59/59 [==============================] - 3s 53ms/step - loss: 0.4335 - accuracy: 0.8159 - val_loss: 0.4280 - val_accuracy: 0.8017
Epoch 3/10
59/59 [==============================] - 3s 56ms/step - loss: 0.2662 - accuracy: 0.9051 - val_loss: 0.4573 - val_accuracy: 0.7867
Epoch 4/10
59/59 [==============================] - 3s 56ms/step - loss: 0.1495 - accuracy: 0.9563 - val_loss: 0.5037 - val_accuracy: 0.8006
Epoch 5/10
59/59 [==============================] - 3s 55ms/step - loss: 0.0892 - accuracy: 0.9740 - val_loss: 0.5537 - val_accuracy: 0.8049
Epoch 6/10
59/59 [==============================] - 3s 57ms/step - loss: 0.0737 - accuracy: 0.9783 - val_loss: 0.6100 - val_accuracy: 0.7931
Epoch 7/10
59/59 [==============================] - 3s 55ms/step - loss: 0.0596 - accuracy: 0.9812 - val_loss: 0.6594 - val_accuracy: 0.7889
Epoch 8/10
59

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load the preprocessed dataset
data = pd.read_excel("preprocessed_output.xlsx")

# Convert lists of strings to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

# Pad sequences to a fixed length
max_sequence_length = 100
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the dataset into training and testing sets
y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# Calculate evaluation metrics
report = classification_report(y_test, y_pred)
print(report)


Epoch 1/10
59/59 [==============================] - 66s 937ms/step - loss: 0.6460 - accuracy: 0.6249 - val_loss: 0.5943 - val_accuracy: 0.6795
Epoch 2/10
59/59 [==============================] - 54s 917ms/step - loss: 0.3745 - accuracy: 0.8365 - val_loss: 0.4409 - val_accuracy: 0.7846
Epoch 3/10
59/59 [==============================] - 23s 398ms/step - loss: 0.2458 - accuracy: 0.9059 - val_loss: 0.5117 - val_accuracy: 0.7985
Epoch 4/10
59/59 [==============================] - 24s 411ms/step - loss: 0.1680 - accuracy: 0.9416 - val_loss: 0.5240 - val_accuracy: 0.7867
Epoch 5/10
59/59 [==============================] - 25s 417ms/step - loss: 0.1197 - accuracy: 0.9579 - val_loss: 0.7356 - val_accuracy: 0.7760
Epoch 6/10
59/59 [==============================] - 22s 364ms/step - loss: 0.0767 - accuracy: 0.9737 - val_loss: 0.7254 - val_accuracy: 0.7749
Epoch 7/10
59/59 [==============================] - 24s 408ms/step - loss: 0.0649 - accuracy: 0.9770 - val_loss: 0.8653 - val_accuracy: 0.7663

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

# Load the preprocessed dataset
data = pd.read_excel("preprocessed_output.xlsx")

# Convert lists of strings to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Tokenized Text"])
sequences = tokenizer.texts_to_sequences(data["Tokenized Text"])

# Pad sequences to a fixed length
max_sequence_length = 100
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the dataset into training and testing sets
y = data["task_1"].apply(lambda x: 1 if x == "HOF" else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the BiLSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# Calculate evaluation metrics
report = classification_report(y_test, y_pred)
print(report)


Epoch 1/10
59/59 [==============================] - 76s 1s/step - loss: 0.6338 - accuracy: 0.6318 - val_loss: 0.4926 - val_accuracy: 0.7674
Epoch 2/10
59/59 [==============================] - 70s 1s/step - loss: 0.3553 - accuracy: 0.8574 - val_loss: 0.4373 - val_accuracy: 0.8060
Epoch 3/10
59/59 [==============================] - 74s 1s/step - loss: 0.2332 - accuracy: 0.9137 - val_loss: 0.6561 - val_accuracy: 0.7889
Epoch 4/10
59/59 [==============================] - 75s 1s/step - loss: 0.1524 - accuracy: 0.9483 - val_loss: 0.6302 - val_accuracy: 0.7814
Epoch 5/10
59/59 [==============================] - 76s 1s/step - loss: 0.1154 - accuracy: 0.9617 - val_loss: 0.7926 - val_accuracy: 0.7663
Epoch 6/10
59/59 [==============================] - 76s 1s/step - loss: 0.0929 - accuracy: 0.9692 - val_loss: 0.9304 - val_accuracy: 0.7738
Epoch 7/10
59/59 [==============================] - 76s 1s/step - loss: 0.0790 - accuracy: 0.9716 - val_loss: 0.9707 - val_accuracy: 0.7556
Epoch 8/10
59/59 [==